In [8]:
#install package manager
!pip install --upgrade pip

Requirement already up-to-date: pip in /anaconda/lib/python3.6/site-packages (19.1.1)


In [9]:
#install error work-around
import sys
sys.path.append("/anaconda/lib/python3.6/site-packages")

In [10]:
#install graphviz
!brew install graphviz
!pip install pygraphviz --install-option="--include-path=/usr/local/include/graphviz/" \
--install-option="--library-path=/usr/local/lib/graphviz"

To reinstall 2.40.1, run `brew reinstall graphviz`
//anaconda/lib/python3.6/site-packages/pip/_internal/commands/install.py:244: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)


In [4]:
#import data science library
from datascience import *

from IPython.display import Image
from bokeh.charts import HeatMap, output_file, show
from bokeh.palettes import *
import pygraphviz as PG
import numpy as np
%matplotlib inline

In [5]:
#read in table
offerings = Table().read_table('offeringsDrum.csv')
#Replaces value "plural, unspecified" with a value of 5
def replace_plural(table, value, column):
    for i in range(len(table[column])):
        if table[column][i] == 'plural, unspecified':
            table[column][i] = value
        else:
            table[column][i] = int(table[column][i])
    return table
offerings = replace_plural(offerings, 5, 6)

#converts 'number' column from a string to an integer
offerings.set_format('number', NumberFormatter)

#removes rows where no god is specified
#offerings = offerings.where('deity', are.not_containing('nan'))

#print table
offerings

section,offering,utensil,material,decoration,count,number,quantity,measurement,type of substance,position,occasion,personnel,deity,day,month,text,additional
1,serqu offering,nan,nan,nan,1,1,nan,nan,assemblage,nan,nan,priest,the great gods,favorable day,auspicious month,TU 44,nan
1,first quality beer,nan,nan,nan,1,1,nan,nan,beer,nan,nan,priest,the great gods,favorable day,auspicious month,TU 44,nan
1,barley bread,nan,nan,nan,1,7,nan,nan,bread,nan,nan,priest,[text damaged],favorable day,auspicious month,TU 44,nan
1,emmer bread,nan,nan,nan,1,7,nan,nan,bread,nan,nan,priest,[text damaged],favorable day,auspicious month,TU 44,nan
1,barley bread,nan,nan,nan,1,7,nan,nan,bread,nan,nan,priest,[text damaged],favorable day,auspicious month,TU 44,nan
1,emmer bread,nan,nan,nan,1,7,nan,nan,bread,nan,nan,priest,[text damaged],favorable day,auspicious month,TU 44,nan
1,mirsu cake,nan,nan,nan,1,1,nan,nan,confection,nan,nan,priest,[text damaged],favorable day,auspicious month,TU 44,"made with honey, ghee, dates, and sisqu flour"
1,shoulder,nan,nan,nan,1,1,nan,nan,meat,nan,nan,priest,[text damaged],favorable day,auspicious month,TU 44,part of the innards
1,himsu omentum,nan,nan,nan,1,1,nan,nan,meat,nan,nan,priest,[text damaged],favorable day,auspicious month,TU 44,nan
1,roasted meat,nan,nan,nan,1,1,nan,nan,meat,nan,nan,priest,[text damaged],favorable day,auspicious month,TU 44,nan


In [12]:
#create pivot table. "Group" is a function that returns a table with each row corresponding to a unique value.
#lamda x: sum(x) takes the summed values of number x count.
#lambda x: np.log(sum(x)) thakes the log of the sum

groupedob = offerings.group(['offering', 'deity'], lambda x: sum(x))

#selects columns to be mapped
groupedob = groupedob.select('offering', 'deity', 'number')

groupedob.sort("number")

#makes groupedob a panda dataframe
groupedob = groupedob.to_df()

groupedob

,offering,deity,number
0,barley bread,[text damaged],14
1,barley loaf,twelve gods,7
2,barley malt,twelve gods,5
3,beer,bull(?),1
4,beer,twelve gods,2
5,emmer bread,[text damaged],14
6,emmer loaf,twelve gods,7
7,emmer malt,twelve gods,5
8,first quality beer,Ea; Shamash; Marduk; Lumha; divine kettledrum,1
9,first quality beer,Lumha,2


In [13]:
#create heat map

hm = HeatMap(groupedob, x='offering', y='deity', values='number', title='Offering and Deity', height=1000, width=1500, stat=None, 
             palette=viridis(12))
                                                    
output_file('drum.html')

show(hm)

INFO:bokeh.core.state:Session output file 'drum.html' already exists, will be overwritten.
